# Pré-processamento de Dados

###### O tratamento dos dados é uma etapa fundamental para garatir a eficácia dos modelos de treinamento
###### Entre as muitas etapas deste processo podemos destacar:


## Lidando com Dados Perdidos

In [2]:
import pandas as pd
from io import StringIO

In [3]:
#gerando dados articiais
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [4]:
#Podemos usar o método isnull para retornar um DataFrame com valores booleanos que indicam 
#se uma célula contém um valor numérico (False) ou se faltam dados (True).
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

#### A abordagem mais simples é  eliminar estes dados, usando a função dropna

In [5]:
df.dropna(axis=1) #axis=1, elimina as colunas com valores vazios (NaN)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [6]:
df.dropna(how= 'all') #Remove as linhas com todos os elementos NaN
df.dropna(thresh=4)   #Remove as linhas com menos que 4 
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


   ### Pode ocorrer da quantidade de valores vazios na tabela ser muito grande. Podemos então usar a função Mean Imputation, onde substituimos os valores nulos pelo valor medio da coluna, usando SimpleImputer

In [8]:
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values= np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

#### Outras opções de strategy são 'median' and 'most_frequent'

### Outra forma mais conveniente é usar 'fillna' do pandas


In [16]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


## Lidando com Dados categoricos

### Os tipos de dados categoricos são: 

* **Ordinal features** são dados que podem ser ordenados: Como tamanho de camisas XL>L> M
* **Nominal features** não implicam em ordem, como as cores 

In [13]:
#gerando dados artificiais
import pandas as pd 
df = pd.DataFrame([
    ['green','M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['glass', 'XL', 15.3, 'class2']
    
])
df.columns =['color', 'size','price', 'classLabel' ]

df

,color,size,price,classLabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,glass,XL,15.3,class2


##### Criamos um dataFrame contendo uma coluna com valores nominais (color), valores ordinais (size) e valores numericos (preço).  Os rótulos das classes (assumindo que criamos um conjunto de dados para um supervisionado tarefa de aprendizagem) são armazenados na última coluna. Os algoritmos de aprendizagem para classificação que discutimos neste livro não usam informações ordinais nos rótulos de classe.

#### Para garantir que os algoritmos de aprendizagem interpretem corretamente, podemos transformar os valores categoricos em strings. Não há um função conveniente  que transforma automaticamente uma ordem correta os labels do rotulos size, isso nós fazemos manualmente assumindo que sabemos as diferença numérica entre as features, como por exemplo: XL = L+1 = M+2:

In [18]:
size_mapping = {'XL':3, 'L': 2, 'M':1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classLabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,glass,3,15.3,class2


In [25]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

## Encoding class labels